## PoC парсера сайта bestceramic.ru

In [1]:
#начинаем с импорта библиотек
import requests
from requests.auth import HTTPProxyAuth # для проксей
from  bs4 import BeautifulSoup as bs
import csv
from urllib.request import urljoin

In [2]:
# Задаем глобальные параметры
headers = {'accept': '*/*',
           'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36'
          }
domen = 'https://www.bestceramic.ru/'
url='https://www.bestceramic.ru/catalog/'
filename = 'product_links.csv'

Функция get_category_links запускается первой. Она собирает ссылки на основные категории.
Парсер работает по выбранной категории т.к. товарных SKU очень много и нет смысла мешать
паркет со смесителями в один файл.

In [ ]:
get_category_links(headers, domen, url)

In [87]:
# Функция сбора ссылок на основные продуктовые категории
def get_category_links(headers, domen, url):
    session = requests.Session()
    r = session.get(url, headers=headers)
    soup = bs(r.text, 'lxml')
    links = []
    urls=[]
    ul = soup.find('div', class_='tile-grid').find_all('a', class_='outside')
    if ul:
        for u in ul:
             urls.append(u['href'])
    for i in urls:
        link  = urljoin(domen,i)
        links.append(link)
    return links

После того, как получили ссылки на товарные категории, выбираем любую для парсинга, например так:

In [ ]:
url - links[7] #выберет шестую по счету категорию для парсинга

После выбора товарной категории запускаем пагинатор и сборщик урлов целевых страниц:

In [ ]:
temp_plinks=[] #для вложенного списка
product_links=[] #для прямого списка
pagi_urls = get_pagination(url, headers) - #получаем список пагинационных урлов с одной категории
for i in pagi_urls:
    a = get_grid_links(i, headers)
    temp_plinks.append(a)  

for sublist in temp_plinks:
    for item in sublist:
        product_links.append(item)   
save_products(product_links) 

In [104]:
#  блок сбора урлов пагинации
def get_pagination(url, headers):
    session = requests.Session()
    r = session.get(url, headers=headers)
    pagi=[] 
    pagi.append(url)
    if r.status_code == 200:
        soup = bs(r.content, 'lxml')
        try:
            count = int(soup.find('div', class_='tile-pages').contents[7].text)
            for i in range(count+1):
                u = str(url)+f'/{i}'
                if u not in pagi:
                    pagi.append(u)
            del pagi[1:3] #удалим нулевую и первую пагинацию
        except:            
            pass
        return pagi

In [40]:
# функция сбора ссылок на целевые страницы.
def get_grid_links(url, headers):
    global domen
    r = session.get(url, headers=headers)
    soup = bs(r.content, 'lxml')
    urls=[]
    product_links=[]
    if soup:
        try:
            a = soup.find_all('div',class_='grid-content-3col')
            for i in a:
                b = soup.find('div', class_='tile-grid').find_all('div', class_='title')
                for i in b:
                    c = i.find('a', href=True)
                    urls.append(c['href'])
        except:
            print('Сбой на этапе чтения тегов')
            pass
    else:
        print('Суп не обнаружен')
        return False # это чтобы остановить работу функции. Т.к. если супа нет, то и склеивать ссылки смысла нет.
    for i in urls:
        link  = urljoin(domen,i)
        product_links.append(link)
    return product_links

In [32]:
def save_products (result):
    with open('product_links.csv', 'w') as outfile:
        spamwriter = csv.writer(outfile, lineterminator='\n')
        spamwriter.writerow(['Product_Link']) 
        for i in result:
            spamwriter.writerow([i])

После отработки в папке с парсером появится файл product_links.csv со ссылками.
Имея сохраненные ссылки в файле нам не нужно запускать парсер каждый раз заново,
в случаем сбоя или если сервер заблокирует бота. Сменив прокси, можно будет начать уже с этого шага.

И финально запускаем уже сам главный модуль такой командой:

In [ ]:
get_info(headers, domen, filename) # - запуск главного модуля парсера

Если все четко - появится сообщение - DONE! и в папке с парсером будет сохранен файл
products.csv со всеми позициями из выбранной категории и их характеристиками

In [43]:
#функция парсинга целевых страниц
def get_info(headers, domen, filename):
    results = open_csv(filename)
    products=[]
    session = requests.Session()
    for url in results:
        params={} 
        r = session.get(url, headers=headers)
        soup = bs(r.text, 'lxml')
        if soup:
            try:
                a=soup.find('h1', class_='title')
                if a:
                    params['title']=a.text
                b=soup.find('div', class_='model')
                if b:
                    params['articul']=b.text.rstrip('Сообщите код оператору').lstrip('Код: ')
                c=soup.find('div', class_='product-body__title')
                if c:
                    params['price']=c.text.strip()
                e=soup.find('div', class_='tab-inner__left-block').find_all('div', class_='prop-row')
                if e:
                    l1=[]
                    l2=[]
                    dict1={}
                    for i in e:
                        ee=i.find('div', class_='prop-title', span='').text.strip()
                        l1.append(ee)
                        eee=i.find('div', class_='prop-value', span='').text.strip()
                        l2.append(eee)
                    dict1 = dict(zip(l1, l2))
                    params.update(dict1)
                params['url']=url
                f=soup.find('div', class_='big').find('a', href=True)['href']
                if f:
                    src = urljoin(domen,f)
                    params['image']=src
                products.append(params)
            except:
                print('Сбой при парсинге тегов')
                pass
        else:
            print('Проблемы с супом')
            return False
    save_product_info(products)
    print('DONE!')

In [3]:
# Функция загрузки файла со ссылками на целевые страницы
def open_csv(filename):
    results=[]
    with open(filename, 'r') as data:
        data.readline().rstrip()
        for line in data:
            results.append(line[:-1])
    return results

In [16]:
# Функция сохранения результатов парсинга целевых страниц
def save_product_info(products):
    with open('products.csv', 'w', encoding = 'utf-8') as outfile:
        spamwriter = csv.DictWriter(outfile, fieldnames=get_field_name(products), lineterminator='\n')    
        spamwriter.writeheader() #записывет заголовки столбцов равные ключам словаря
        spamwriter.writerows(products)

In [6]:
# Функция сбора ключей словаря для формирования хэдера в CSV
def get_field_name(products):
    field_names = set()
    for i in products:
        for k in i:
            field_names.add(k)
    return field_names